# BSS-AUS: Basketball Statistic System (AUS)

This system allows to **incrementally** build a database of _stint lineups_ (advance) statistics for each game and each team.

A **stint** is a lineup of players who play together in different interval periods across the game. The system will build the stints for each team from the play-by-play data and compute various statistics.

The data comes as a raw JSON file:

https://fibalivestats.dcd.shared.geniussports.com/data/2087737/data.json

In [1]:
# Let's first load all required packages...
import os
import pandas as pd
import numpy as np
import dtale

from config import *
import bball_stats
import tools

# If one wants Jupyter to report all vars, not just the last one
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


## 1. Define games to scrape and saved data

First, setup the games we want to scrape and compute, as well as the existing data stored in file to append to.

In [2]:
# games to be computed
# format: (game id, game no for team 1, game no for team 2)
games = [(1999318, 1, 1), (2087737, 2, 2), (2090351, 3, 3)]
games = [(1976463, 0, 0)]

games = [(1976446,1,1,1,1),(1976447,1,1,1,1),(1976448,1,1,1,1),(1976452,2,1,1,1),(1976454,1,1,1,1),(2004608,2,1,1,1),(2004609,3,2,2,2),(1976449,2,2,2,2),(1976451,2,2,3,2),(1976453,2,2,3,2),(1976455,4,2,3,2),(1976458,2,2,2,2),(1976456,2,2,3,2),(1976457,3,3,3,3),(1976459,3,3,4,3),(1976460,4,3,3,3),(1976461,4,3,4,3),(1976462,3,3,5,3),(1976463,4,3,5,3),(1976464,4,3,4,3),(2004610,5,3,4,3),(1976465,5,3,5,3),(1976468,6,4,6,4),(1976469,6,4,5,4),(1976474,7,5,6,5),(1976473,4,5,6,5),(1976482,5,6,7,6),(2036215,5,7,7,7),(2031329,8,7,7,7),(2031330,6,7,5,7),(2031332,6,7,8,7),(2031333,8,7,9,7),(2031334,6,7,7,7),(2031335,7,8,9,8),(2031336,7,8,9,8),(2031337,7,8,8,8),(2031338,10,8,8,8),(2031340,8,8,6,8),(2031341,7,8,5,8),(2046695,8,8,8,8),(2046696,9,8,10,8),(2046697,9,9,9,9),(2031342,9,9,9,9),(2031343,6,9,10,9),(2031344,10,9,8,9),(2031345,10,9,11,9),(2031346,11,9,10,9),(2031347,10,9,10,9),(2046698,11,10,11,10),(2046700,11,10,12,10),(2046701,11,10,7,10),(2046702,9,10,11,10),(2046703,13,10,12,10),(2046704,12,10,12,10),(2046706,12,10,10,10),(2046707,11,11,14,11),(2046709,12,11,11,11),(2046710,13,11,8,11),(2046711,13,11,12,11),(2046712,12,11,13,11),(2046713,15,11,13,11),(2051763,9,11,13,11),(2053811,14,12,13,12),(2053812,14,12,10,12),(2053813,14,12,14,12),(2053814,16,12,13,12),(2053815,12,12,11,12),(2053816,15,12,14,12),(2053817,15,12,14,12),(2053818,12,13,14,12),(2053819,16,13,13,13),(2053820,16,13,15,13),(2053821,15,13,14,13),(2053822,14,13,17,13),(2053823,13,13,16,13),(2053824,16,13,15,13),(2053825,15,13,17,13),(2056454,15,14,16,14),(2056455,17,14,16,14),(2056457,17,14,17,14),(2056458,16,14,16,14),(2056461,17,14,18,14),(2056462,17,14,18,14),(2056460,18,14,14,14),(2056463,18,15,17,15),(2056464,15,15,18,15),(2056466,19,15,18,15),(2056467,18,15,17,15),(2056469,19,15,19,15),(2056471,18,15,19,15),(2056472,16,15,19,15),(2056473,19,15,19,15),(2065653,20,16,19,16),(2065654,18,16,17,16),(2065655,20,16,21,16),(2065656,20,16,20,16),(2065657,21,17,19,16),(2065658,21,17,18,16),(2065659,19,17,20,16),(2069165,20,16,21,17),(2069166,20,16,21,17),(2069167,20,16,21,17),(2069168,22,17,22,17),(2069169,22,17,21,17),(2069170,20,17,22,17),(2069171,21,17,22,17),(2069172,22,17,20,17),(2069175,23,18,23,18),(2069177,21,18,23,18),(2069179,22,18,24,18),(2069181,21,18,23,18),(2069183,24,18,22,18),(2069184,24,18,22,18),(2069186,24,18,22,18),(2069187,23,18,23,18),(2069191,23,18,24,19),(2069192,24,19,24,19),(2069194,24,19,23,19),(2069196,25,19,25,19),(2069199,25,19,23,19),(2069202,23,19,25,19),(2069203,25,19,25,19),(2069204,24,19,26,19),(2069190,24,19,26,20),(2069193,26,19,27,20),(2069195,26,20,25,20),(2069197,25,20,27,20),(2069198,27,20,26,20),(2069200,26,20,25,20),(2069201,26,20,25,20),(2069205,26,20,24,19),(2069173,26,20,28,21),(2069174,28,21,28,21),(2069176,28,21,28,21),(2069178,28,21,28,21),(2069180,28,21,28,21),(2069182,28,21,27,21),(2069188,27,21,27,21),(2069189,27,20,26,20)]

games = [1976463]   # game with no "bugs" in subs

# Files containing the existing tables stored in disk already
# System will extend these tables with the new games
file_stats_df = os.path.join(DATA_DIR, "stats_df.pkl")
file_games_df = os.path.join(DATA_DIR, "games_df.pkl")

# Set to true to re-compute from scratch all tables
reload = True

## 2. Compute stat and game tables

Now, let us run the system that scrapes the games' data, computes stats and game info, and adds them to the initial tables of stats and games.

We start by loading all saved previous games, if any, as we want to append to that database (and we don't want to recompute them).

In [3]:
# Load games from saved files (if any)
init_stats_df = None
game_df = None
if os.path.exists(file_stats_df) and not reload:
    # load the stat dataframe already stored as a file
    print(f"Loading initial stats df: {file_stats_df}")
    init_stats_df = pd.read_pickle(file_stats_df)
    game_df = pd.read_pickle(file_games_df)
    existing_games = init_stats_df.game_id.unique()
else:
    existing_games = []


init_stats_df
# init_stats_df['lineup'].apply(lambda x: len(x) > 5)
# init_stats_df.loc[5,'lineup']
# init_stats_df.loc[5]

It is now time to process each game:

In [7]:
stats_dfs = [init_stats_df] if init_stats_df is not None else []
games_data = []
for game in games:
    if isinstance(game, tuple):
        game_id, game1, round1, game2, round2 = game
    else:
        game_id = game
        game1 = round1 = game2 = round2  = np.nan

    if game_id in existing_games:
        print(f"Game {game_id} is already in table")
        continue
    print(f"Computing game {game_id}...")

    # now compute the actual stats for the game
    result = bball_stats.build_game_stints_stats_df(game_id)
    df = result['stint_stats_df']   #  this is basically what we care, the stint stats
    team1, team2 = result['teams']

    try:
        game_info = tools.get_game_info(game_id)    # extract date of game from HTML
    except:
        game_info = { "venue" : np.nan, "date": np.nan}
    print(f"\t .... done: {team1[0]} ({team1[1]}) vs {team2[0]} ({team2[1]}) on {game_info['date']}")

    # fill game info
    df.insert(0, 'game_id', game_id)
    df.insert(3, 'game1', game1)
    df.insert(4, 'round1', round1)
    df.insert(5, 'game2', game2)
    df.insert(6, 'round2', round2)
    stats_dfs.append(df)

    # build game dataframe table
    games_data.append({"game_id": game_id,
                        "date" : game_info['date'],
                        "team1": team1[0], "team2": team2[0],
                        "s1": team1[1], "s2": team2[1],
                        "game1": game1, "round1": round1,
                        "game2": game2, "round2": round2,
                        "winner": 1 if team1[1] > team2[1] else 2,
                        "venue" : game_info["venue"]}
                      )

# put all dfs together into a single dataframe
stats_df = pd.concat(stats_dfs)
stats_df.reset_index(inplace=True, drop=True)
stats_df.sample(5)

if game_df is not None:
    games_df = pd.concat([game_df, pd.DataFrame(games_data)])
    games_df.reset_index(inplace=True, drop=True)
else:
    games_df = pd.DataFrame(games_data)

Computing game 1976463...
	 .... done: Melbourne United (83) vs New Zealand Breakers (60) on 2021-12-19 00:00:00


If we want we can do some sanity checks, before saving to disk:

In [ ]:
games_df

In [8]:
print("The shape of stats_df is:", stats_df.shape)
stats_df.sample(5)
dtale.show(stats_df)
# stats_df.loc[4]

The shape of stats_df is: (34, 99)


In [ ]:
# (ortg, drtg) should mirror (drtg_opp, ortg)
stats_df.iloc[41][['poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]]

## 3. Save stats and games to files

We now save the full dataframes (stats and games) in various formats: binary (pickle), csv, and Excel.

This will allows us to re-load that data later to add more games to it quicker.

In [ ]:
import datetime
import os

stats_df.to_pickle(os.path.join(DATA_DIR, "stats_df.pkl"))
games_df.to_pickle(os.path.join(DATA_DIR, "games_df.pkl"))

stats_df.to_csv(os.path.join(DATA_DIR, "stats_df.csv"), index=False)
games_df.to_csv(os.path.join(DATA_DIR, "games_df.csv"), index=False)

with pd.ExcelWriter(os.path.join(DATA_DIR, 'stats_df.xlsx')) as writer:
    stats_df.to_excel(writer, sheet_name='STATS', index=False)
    games_df.to_excel(writer, sheet_name='GAMES', index=False)
games_df.to_excel(os.path.join(DATA_DIR, "games_df.xlsx"))


now = datetime.datetime.now() # current date and time
date_time = now.strftime("%m/%d/%Y, %H:%M:%S")
print(f"Finished saving at time: {date_time}")

## 4. Inspection & analysis

We use [dtale](https://pypi.org/project/dtale/) package for this.

In [ ]:
dtale.show(stats_df)
# dtale.show(stats_df[['tno', 'stint', 'poss', 'ortg', 'drtg', "poss_opp", "ortg_opp", "drtg_opp"]])

## 5. Some checks...

Check if a stint lineup has more than 5 players! It could happen:

1. Game 2031329, player H. Besson comes out (wrongly?) at 3rd period min 10:00 but he keeps playing and then goes out again at 7:33.

In [ ]:
stats_df.shape
mask = stats_df['lineup'].apply(lambda x: len(x) != 5)
stats_df[mask]

# stats_df.iloc[941][['game_id', 'lineup']]